# Evaluation and Comparison on Feedback Dataset

In [1]:
# Setup and Import Libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    roc_curve, auc, confusion_matrix, classification_report
import seaborn as sns

sys.path.append('..')
from attention_model import SimpleAttentionCNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Model Paths and Loading

In [2]:
# Define paths
ORIGINAL_MODEL_PATH = "../model/tb_chest_xray_attention_best.pt"
FINETUNED_MODEL_PATH = "../finetuning/tb_chest_xray_refined_20250318_034728.pt"
FEEDBACK_IMG_DIR = "../feedback/images"
FEEDBACK_CSV = "../feedback/feedback_log.csv"

# Check if the files and directories exist
print(f"Original model exists: {os.path.exists(ORIGINAL_MODEL_PATH)}")
print(f"Finetuned model exists: {os.path.exists(FINETUNED_MODEL_PATH)}")
print(f"Feedback images directory exists: {os.path.exists(FEEDBACK_IMG_DIR)}")
print(f"Feedback log exists: {os.path.exists(FEEDBACK_CSV)}")

# Load model
def load_model(model_path):
    model = SimpleAttentionCNN().to(device)
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Successfully loaded model from {model_path}")
        return model
    except Exception as e:
        print(f"Error loading model from {model_path}: {e}")
        return None

# Load both models
original_model = load_model(ORIGINAL_MODEL_PATH)
finetuned_model = load_model(FINETUNED_MODEL_PATH)

# Set models to evaluation mode
if original_model:
    original_model.eval()
if finetuned_model:
    finetuned_model.eval()

## 2. Load Feedback Dataset

We'll load the feedback dataset using information from the feedback log CSV.

In [3]:
def load_feedback_dataset():
    """Load the feedback dataset from feedback_log.csv"""
    if not os.path.isfile(FEEDBACK_CSV) or not os.path.isdir(FEEDBACK_IMG_DIR):
        print(f"Feedback data not found: {FEEDBACK_CSV} or {FEEDBACK_IMG_DIR}")
        return None, None
        
    try:
        # Read the CSV
        df = pd.read_csv(FEEDBACK_CSV, header=None)
        
        # Check if we need to add column names
        if df.shape[1] >= 4:  # Expecting at least 4 columns
            df.columns = ['image_filename', 'mask_filename', 'label', 'timestamp']
        elif df.shape[1] == 3:
            df.columns = ['image_filename', 'mask_filename', 'label']
        
        paths = []
        labels = []
        
        for _, row in df.iterrows():
            img_file = row['image_filename']
            label_str = row['label']
            
            # Create full path to image
            img_path = os.path.join(FEEDBACK_IMG_DIR, os.path.basename(img_file))
            
            if os.path.isfile(img_path):
                # Convert label: TB -> 1, Normal -> 0
                label = 1 if label_str.upper() == 'TB' else 0
                paths.append(img_path)
                labels.append(label)
        
        print(f"Loaded {len(paths)} feedback images")
        print(f"Class distribution: {labels.count(0)} Normal, {labels.count(1)} TB")
        
        return paths, labels
    
    except Exception as e:
        print(f"Error loading feedback data: {e}")
        return None, None

# Load feedback dataset
feedback_paths, feedback_labels = load_feedback_dataset()

if feedback_paths is None:
    print("Failed to load feedback dataset!")

## 3. Prepare Feedback Dataset for Testing

In [4]:
class FeedbackDataset(Dataset):
    """Dataset class for the feedback images."""
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform or T.Compose([
            T.Resize((256, 256)),
            T.ToTensor()
        ])
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        
        try:
            # Read the image
            img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img_gray is None:
                raise FileNotFoundError(f"Could not read image: {img_path}")
            
            # Convert to PIL for transforms
            pil_img = Image.fromarray(img_gray, mode='L')
            img_tensor = self.transform(pil_img)
            
            return img_tensor, torch.tensor(label, dtype=torch.float32), img_path
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")
            # Return empty tensor with same shape
            return torch.zeros(1, 256, 256), torch.tensor(label, dtype=torch.float32), img_path

# Create dataset and dataloader for the feedback data
if feedback_paths is not None and len(feedback_paths) > 0:
    # Define transforms that match the original model's preprocessing
    test_transforms = T.Compose([
        T.Resize((256, 256)),
        T.ToTensor()
    ])
    
    feedback_dataset = FeedbackDataset(feedback_paths, feedback_labels, transform=test_transforms)
    feedback_dataloader = DataLoader(feedback_dataset, batch_size=16, shuffle=False)
    
    print(f"Created feedback dataset with {len(feedback_dataset)} images")
else:
    print("Could not create feedback dataset!")

## 4. Model Evaluation Functions

In [5]:
def evaluate_model(model, dataloader, model_name="Model"):
    """Evaluate model and calculate all metrics."""
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    all_attn_maps = []
    all_img_paths = []
    
    with torch.no_grad():
        for batch in dataloader:
            images, labels, paths = batch
            images = images.to(device)
            
            # Forward pass
            outputs, attention_maps = model(images)
            
            # Get predictions
            probs = outputs.cpu().numpy().flatten()
            preds = (outputs >= 0.5).cpu().numpy().flatten().astype(int)
            
            # Store results
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs)
            all_attn_maps.extend([attn.cpu().numpy() for attn in attention_maps])
            all_img_paths.extend(paths)
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    f1 = f1_score(all_labels, all_preds, average='binary')
    cm = confusion_matrix(all_labels, all_preds)
    
    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(all_labels, all_probs)
    roc_auc = auc(fpr, tpr)
    
    # Print metrics
    print(f"\n--- {model_name} Performance Metrics on Feedback Dataset ---")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-ROC: {roc_auc:.4f}")
    print("Confusion Matrix:")
    print(cm)
    
    results = {
        'preds': all_preds,
        'labels': all_labels,
        'probs': all_probs,
        'attention_maps': all_attn_maps,
        'image_paths': all_img_paths,
        'metrics': {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc': roc_auc,
            'fpr': fpr,
            'tpr': tpr,
            'confusion_matrix': cm
        }
    }
    
    return results

## 5. Evaluate Both Models on Feedback Dataset

In [6]:
# Run evaluation for both models on feedback data
model_results = {}

if 'feedback_dataloader' in locals() and original_model:
    print("Evaluating original model on feedback dataset...")
    model_results['original'] = evaluate_model(original_model, feedback_dataloader, "Original Model")
else:
    print("Original model or feedback data not available. Cannot evaluate.")

if 'feedback_dataloader' in locals() and finetuned_model:
    print("\nEvaluating finetuned model on feedback dataset...")
    model_results['finetuned'] = evaluate_model(finetuned_model, feedback_dataloader, "Finetuned Model")
else:
    print("Finetuned model or feedback data not available. Cannot evaluate.")

## 6. Visualization of Results

In [7]:
def visualize_confusion_matrix(cm, class_names=['Normal', 'Tuberculosis'], title='Confusion Matrix'):
    """Visualize confusion matrix as a heatmap."""
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.tight_layout()
    plt.show()

def plot_roc_curves(results_dict):
    """Plot ROC curves for multiple models on same graph."""
    plt.figure(figsize=(10, 7))
    
    colors = {'original': 'blue', 'finetuned': 'red'}
    labels = {'original': 'Original Model', 'finetuned': 'Finetuned Model'}
    
    for model_name, results in results_dict.items():
        if model_name in results_dict:
            fpr = results['metrics']['fpr']
            tpr = results['metrics']['tpr']
            roc_auc = results['metrics']['auc']
            color = colors.get(model_name, 'green')
            label = labels.get(model_name, model_name)
            plt.plot(fpr, tpr, color=color, lw=2,
                     label=f'{label} (AUC = {roc_auc:.4f})')
    
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves Comparison on Feedback Dataset')
    plt.legend(loc='lower right')
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_comparative_metrics_bar_chart(results_dict):
    """Create side-by-side bar chart comparing metrics between models."""
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC-ROC']
    
    # Extract metrics for each model
    metrics_values = {}
    for model_name, results in results_dict.items():
        metrics_values[model_name] = [results['metrics'][metric] for metric in metrics_to_plot]
    
    # Create bar chart
    plt.figure(figsize=(14, 7))
    
    # Set width of bars
    bar_width = 0.35
    index = np.arange(len(metric_labels))
    
    # Create bars
    if 'original' in metrics_values:
        plt.bar(index - bar_width/2, metrics_values['original'], bar_width, 
                label='Original Model', color='cornflowerblue', alpha=0.8)
        
        # Add text labels for original model
        for i, v in enumerate(metrics_values['original']):
            plt.text(i - bar_width/2 - 0.05, v + 0.01, f'{v:.3f}', 
                    color='navy', fontweight='bold', fontsize=9)
    
    if 'finetuned' in metrics_values:
        plt.bar(index + bar_width/2, metrics_values['finetuned'], bar_width,
                label='Finetuned Model', color='lightcoral', alpha=0.8)
        
        # Add text labels for finetuned model
        for i, v in enumerate(metrics_values['finetuned']):
            plt.text(i + bar_width/2 - 0.05, v + 0.01, f'{v:.3f}',
                    color='darkred', fontweight='bold', fontsize=9)
    
    plt.ylabel('Score')
    plt.title('Model Performance Comparison on Feedback Dataset')
    plt.xticks(index, metric_labels)
    plt.ylim(0, 1.1)
    plt.legend(loc='upper right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

In [8]:
# Visualize results for both models
if len(model_results) > 0:
    # 1. Visualize Confusion Matrices for both models
    print("\n--- Confusion Matrix Visualizations ---")
    
    if 'original' in model_results:
        visualize_confusion_matrix(
            model_results['original']['metrics']['confusion_matrix'],
            title='Confusion Matrix - Original Model (Feedback Dataset)'
        )
    
    if 'finetuned' in model_results:
        visualize_confusion_matrix(
            model_results['finetuned']['metrics']['confusion_matrix'],
            title='Confusion Matrix - Finetuned Model (Feedback Dataset)'
        )
    
    # 2. Plot comparative ROC Curve
    print("\n--- ROC Curves Comparison ---")
    plot_roc_curves(model_results)
    
    # 3. Plot comparative bar chart of metrics
    print("\n--- Performance Metrics Comparison ---")
    plot_comparative_metrics_bar_chart(model_results)

## 7. Analysis of Prediction Differences

In [9]:
def compare_predictions(results_dict):
    """Compare prediction differences between models."""
    if 'original' not in results_dict or 'finetuned' not in results_dict:
        print("Need both models to compare predictions.")
        return
        
    orig_preds = results_dict['original']['preds']
    ft_preds = results_dict['finetuned']['preds']
    labels = results_dict['original']['labels']  # Labels are the same for both models
    img_paths = results_dict['original']['image_paths']
    
    # Find indices where models differ in predictions
    diff_indices = [i for i in range(len(orig_preds)) if orig_preds[i] != ft_preds[i]]
    
    # Classification of differences
    improved = [i for i in diff_indices if ft_preds[i] == labels[i] and orig_preds[i] != labels[i]]
    worsened = [i for i in diff_indices if ft_preds[i] != labels[i] and orig_preds[i] == labels[i]]
    
    print(f"Total feedback samples: {len(orig_preds)}")
    print(f"Samples where models predict differently: {len(diff_indices)}")
    print(f"Improvements (finetuned correct, original wrong): {len(improved)}")
    print(f"Regressions (finetuned wrong, original correct): {len(worsened)}")
    
    # Display examples of improvements
    if improved:
        print("\nExamples of improvements in feedback dataset:")
        plt.figure(figsize=(15, 4*min(3, len(improved))))
        
        for i, idx in enumerate(improved[:min(3, len(improved))]):
            img_path = img_paths[idx]
            true_label = "Tuberculosis" if labels[idx] == 1 else "Normal"
            orig_pred = "Tuberculosis" if orig_preds[idx] == 1 else "Normal"
            ft_pred = "Tuberculosis" if ft_preds[idx] == 1 else "Normal"
            
            # Load image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                plt.subplot(min(3, len(improved)), 1, i+1)
                plt.imshow(img, cmap='gray')
                plt.title(f"True: {true_label}, Original: {orig_pred}, Finetuned: {ft_pred}\nFile: {os.path.basename(img_path)}")
                plt.axis('off')
        
        plt.tight_layout()
        plt.show()
    
    # Display examples of regressions
    if worsened:
        print("\nExamples of regressions in feedback dataset:")
        plt.figure(figsize=(15, 4*min(3, len(worsened))))
        
        for i, idx in enumerate(worsened[:min(3, len(worsened))]):
            img_path = img_paths[idx]
            true_label = "Tuberculosis" if labels[idx] == 1 else "Normal"
            orig_pred = "Tuberculosis" if orig_preds[idx] == 1 else "Normal"
            ft_pred = "Tuberculosis" if ft_preds[idx] == 1 else "Normal"
            
            # Load image
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                plt.subplot(min(3, len(worsened)), 1, i+1)
                plt.imshow(img, cmap='gray')
                plt.title(f"True: {true_label}, Original: {orig_pred}, Finetuned: {ft_pred}\nFile: {os.path.basename(img_path)}")
                plt.axis('off')
        
        plt.tight_layout()
        plt.show()

In [10]:
# Compare predictions between models on the feedback dataset
if 'original' in model_results and 'finetuned' in model_results:
    print("\n--- Prediction Analysis on Feedback Dataset ---")
    compare_predictions(model_results)
else:
    print("\nNeed both models to compare predictions.")

## 8. Detailed Classification Reports

In [11]:
# Generate classification reports for both models
if 'original' in model_results:
    # Original model report
    print("\n--- Original Model Classification Report (Feedback Dataset) ---")
    orig_report = classification_report(
        model_results['original']['labels'], 
        model_results['original']['preds'],
        target_names=['Normal', 'Tuberculosis']
    )
    print(orig_report)
    
    # Create DataFrame for better visualization
    orig_report_dict = classification_report(
        model_results['original']['labels'],
        model_results['original']['preds'],
        target_names=['Normal', 'Tuberculosis'],
        output_dict=True
    )
    print("Original Model:")
    display(pd.DataFrame(orig_report_dict).transpose().round(4))

if 'finetuned' in model_results:
    # Finetuned model report
    print("\n--- Finetuned Model Classification Report (Feedback Dataset) ---")
    ft_report = classification_report(
        model_results['finetuned']['labels'], 
        model_results['finetuned']['preds'],
        target_names=['Normal', 'Tuberculosis']
    )
    print(ft_report)
    
    # Create DataFrame for better visualization
    ft_report_dict = classification_report(
        model_results['finetuned']['labels'],
        model_results['finetuned']['preds'],
        target_names=['Normal', 'Tuberculosis'],
        output_dict=True
    )
    print("Finetuned Model:")
    display(pd.DataFrame(ft_report_dict).transpose().round(4))